In [1]:
import logging
import math
from dataclasses import dataclass, field
from decimal import ROUND_DOWN, getcontext
from fractions import Fraction
from typing import Any, Optional

import numpy as np
import pandas as pd

In [8]:

cvrs = pd.read_excel("PCC 11.xlsx", usecols=[0, *range(3, 14)], index_col=0)
# guess candidates from all votes, since not given.
candidates = list(set(cvrs.stack().unique()) - {'overvote', 'undervote', np.nan})

array(['overvote', 'Buxton, Catherine A.', 'Chann, Marpheen S.',
       'DiMillo, Steven A.', 'Sheikh-Yousef, Nasreen A.',
       'Condrey, Lawson T.', 'undervote', 'Grant, Benjamin K.',
       'Houseal, Ian P.', 'Bailey, William M.', 'Washburn, Patricia J.',
       'Emerson, Anthony M.', 'Rovelto, Hope R.'], dtype=object)

{'Bailey, William M.',
 'Buxton, Catherine A.',
 'Chann, Marpheen S.',
 'Condrey, Lawson T.',
 'DiMillo, Steven A.',
 'Emerson, Anthony M.',
 'Grant, Benjamin K.',
 'Houseal, Ian P.',
 'Rovelto, Hope R.',
 'Sheikh-Yousef, Nasreen A.',
 'Washburn, Patricia J.'}

In [21]:
candidates = set(cvrs.stack().unique()) - {'overvote', 'undervote', np.nan}

array(['overvote', 'undervote', 'Buxton, Catherine A.',
       'Chann, Marpheen S.', 'Sheikh-Yousef, Nasreen A.',
       'Emerson, Anthony M.', 'Washburn, Patricia J.',
       'DiMillo, Steven A.', 'Condrey, Lawson T.', 'Grant, Benjamin K.',
       'Bailey, William M.', 'Houseal, Ian P.', 'Rovelto, Hope R.'],
      dtype=object)

In [6]:
is_exhausted = pd.Series(np.zeros(cvrs.index.size), index=cvrs.index, dtype=bool)
is_exhausted

Cast Vote Record
22       False
23       False
24       False
25       False
26       False
         ...  
20007    False
20008    False
20009    False
20010    False
20011    False
Length: 9043, dtype: bool

In [ ]:
@dataclass
class TabulationState:
    rounds_completed: int
    cvrs: pd.DataFrame
    is_exhausted: pd.Series | None = None 
    vote_threshold: Any = None
    vote_totals: pd.Series | None = None
    winners: list[str] = field(default_factory=list)
    losers: list[str] = field(default_factory=list)
    transferHistory: list[pd.Series] = field(default_factory=list)


@dataclass
class TabulationRoundSummary:
    round_number: int
    vote_threshold: Any = None
    vote_totals: pd.Series | None = None
    winners: list[str] = field(default_factory=list)
    losers: list[str] = field(default_factory=list)
    transfers: Optional[pd.Series] = None

    def to_dict(self):
        return {
            "totals": self.totals,
            "winners": self.winners,
            "losers": self.losers,
            "transfers": {
                winner: self.transfers[winner].to_dict()
                for winner in self.transfers.labels[0]
            },
        }

def calculate_threshold():
    pass

def tabulate(cvrs):
    is_exhausted = pd.Series(np.zeros(cvrs.index.size), index=cvrs.index, dtype=bool)
    cur_choice_i = pd.Series(np.zeros(cvrs.index.size), index=cvrs.index)



def tabulate_round(tabulation, cvrs, is_exhausted, cur_choices):

    self.rules.skip_to_next_eligible_rank(
        tabulation.ballot_set, tabulation.winners + tabulation.losers
    )

    round = TabulationRound(1 + len(tabulation.rounds))

    round.vote_threshold = calculate_threshold()
    # round.vote_threshold = self.rules.threshold(tabulation)
    round.vote_totals = tabulation.ballot_set.tally_votes()
    round.winners = self.rules.select_winners(round)
    if round.winners:
        for winner in round.winners:
            T = self.rules.T
            excess_votes = round.vote_totals[winner] - round.vote_threshold
            excess_vote_ratio = T(T(excess_votes) / T(round.vote_totals[winner]))
            picked_winner = tabulation.ballot_set.cur_candidate == winner
            vr = tabulation.ballot_set.vote_remaining
            tabulation.ballot_set.df.loc[
                picked_winner, "vote_remaining"
            ] *= excess_vote_ratio

        round.transfers = self.transfer_ballots(
            tabulation.ballot_set,
            round.winners,
            ignore_candidates=tabulation.winners + tabulation.losers,
        )
    else:
        round.losers = self.rules.select_losers(round)
        round.transfers = self.transfer_ballots(
            tabulation.ballot_set,
            round.losers,
            ignore_candidates=round.losers + tabulation.winners + tabulation.losers,
        )
    return round